In [120]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import geopandas as gpd
import urllib
import glob
import matplotlib.pyplot as plt
from fiona.crs import from_epsg
from shapely.geometry import Point, LineString
import scipy

%matplotlib inline

In [2]:
!mkdir download_data
!wget http://streetscore.media.mit.edu/static/files/streetscore_data.zip -O streetscore_data.zip
urllib.request.urlretrieve('https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=Shapefile', 'download_data/precinct.zip')
urllib.request.urlretrieve('http://www1.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/c-summonses/criminal-court-summons-2007-through-q1-2017.xlsx', 'download_data/crime.xlsx')
!mv streetscore_data.zip download_data
zf = ZipFile('download_data/streetscore_data.zip')
zf2 = ZipFile('download_data/precinct.zip')
zf.extractall('download_data/')
zf2.extractall('download_data/')

mkdir: cannot create directory ‘download_data’: File exists
--2018-12-10 12:21:53--  http://streetscore.media.mit.edu/static/files/streetscore_data.zip
Resolving streetscore.media.mit.edu (streetscore.media.mit.edu)... 18.85.28.79
Connecting to streetscore.media.mit.edu (streetscore.media.mit.edu)|18.85.28.79|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4754382 (4.5M) [application/zip]
Saving to: ‘streetscore_data.zip’

streetscore_data.zi 100%[===================>]   4.53M  11.4MB/s    in 0.4s    

2018-12-10 12:21:53 (11.4 MB/s) - ‘streetscore_data.zip’ saved [4754382/4754382]



In [253]:
# ss refers to safety score
ss = pd.read_csv('download_data/streetscore_dataset/streetscore_newyorkcity.csv')
ss.head()

,latitude,longitude,q-score
0,40.700909,-74.013504,11.062166
1,40.701000,-74.013878,10.818611
2,40.701080,-74.012878,12.677955
3,40.701187,-74.013268,11.417325
4,40.701244,-74.012115,25.199091


In [254]:
geometry = [Point(xy) for xy in zip(ss.longitude, ss.latitude)]
linkNYC = ss.drop(['longitude', 'latitude'], axis=1)
crs = {'init': 'epsg:4326'}
ssshp = gpd.GeoDataFrame(ss, crs=crs, geometry=geometry)

In [242]:
#cs refers to crime statistics
cs = pd.read_excel('download_data/crime.xlsx', skiprows=[0,1,2])

In [243]:
cs['index'] = list(cs.index)
precinct_name = cs['Precinct'].unique()

In [244]:
precinct_name

array(['CITYWIDE', nan, '001', '005', '006', '007', '009', '010', '013',
       '014', '017', '018', '019', '020', '022', '023', '024', '025',
       '026', '028', '030', '032', '033', '034', '040', '041', '042',
       '043', '044', '045', '046', '047', '048', '049', '050', '052',
       '060', '061', '062', '063', '066', '067', '068', '069', '070',
       '071', '072', '073', '075', '076', '077', '078', '079', '081',
       '083', '084', '088', '090', '094', '100', '101', '102', '103',
       '104', '105', '106', '107', '108', '109', '110', '111', '112',
       '113', '114', '115', '120', '121', '122', '123', 'Citywide Total'], dtype=object)

In [245]:

def write_precinct(x):
    if x['index'] < 249:
        return precinct_name[0]
    elif (x['index'] % 249 == 0) and (x['index'] != 0):
        return 'total %s'%precinct_name[int(x['index'] / 250)]
    else:
        return precinct_name[int(x['index'] / 250) + 1] 

def rename_precinct(x):
    if x['Precinct'][1] == '0':
        return x['Precinct'][2]
    else:
        if x['Precinct'][0] == 0:
            return x['Precinct'][1:]
        else:
            return x['Precinct']
cs['Precinct'] = cs.apply(write_precinct, axis=1)
cs['precinct'] = cs.apply(rename_precinct, axis=1)

del cs['precinct']

In [246]:
cs.tail()

,Precinct,Violation,Law,CY2007,CY2008,CY2009,CY2010,CY2011,CY2012,CY2013,CY2014,CY2015,CY2016,2017 YTD 03-31,index
19496,123,VIOLATE RESTRICTED USE LICENSE,VTL,0,0,0,0,0,1,0,0,0,0,0,19496
19497,123,WINDSHIELD WASHERS,Traffic Regs,0,0,0,0,0,0,0,0,0,0,0,19497
19498,123,WRONG WAY,Traffic Regs,0,0,0,0,0,0,0,0,0,0,0,19498
19499,123,Total(Violation),NaN,1459,1427,1244,1566,2160,1921,1424,1690,911,1337,166,19499
19500,Citywide Total,NaN,NaN,539800,510962,544678,536021,490604,476801,424883,359537,297413,267763,52310,19500


In [247]:
# pp refers to police precinct
ppshp = gpd.read_file(glob.glob('download_data/*.shp')[0])
ppshp['precinct'] = ppshp['precinct'].astype('int')
ppshp.head()

,precinct,shape_area,shape_leng,geometry
0,1,4.730176e+07,80586.154615,(POLYGON ((-74.0438776157395 40.69018767637665...
1,5,1.808880e+07,18676.124259,POLYGON ((-73.98863862848766 40.72293372026369...
2,6,2.213214e+07,27175.185625,POLYGON ((-73.99968392160721 40.73855224865976...
3,71,4.533179e+07,29978.094261,POLYGON ((-73.92854313809303 40.66457328584737...
4,72,1.046213e+08,87968.194520,POLYGON ((-73.99840899113158 40.67186872303234...


In [248]:
ssshp.head()

,q-score,geometry,precinct
0,11.062166,POINT (-74.013504 40.700909),1
1,10.818611,POINT (-74.01387800000001 40.701),1
2,12.677955,POINT (-74.012878 40.70108),1
3,11.417325,POINT (-74.01326800000001 40.701187),1
4,25.199091,POINT (-74.01211500000001 40.701244),1


In [255]:
# as the safety score was collected in 2011, so first explore the crime statistics in 2011
cs_11 = cs[['Precinct', 'Violation', 'CY2011']]

#put street score and police precinct together

ssshp = gpd.sjoin(ssshp, ppshp, how='left', op='intersects')

In [257]:
ssshp = ssshp.dropna()
ssshp['precinct'] = ssshp['precinct'].astype('int')
ssshp = ssshp[['q-score', 'geometry', 'precinct']]
ssshp.head()

,q-score,geometry,precinct
0,11.062166,POINT (-74.013504 40.700909),1
1,10.818611,POINT (-74.01387800000001 40.701),1
2,12.677955,POINT (-74.012878 40.70108),1
3,11.417325,POINT (-74.01326800000001 40.701187),1
4,25.199091,POINT (-74.01211500000001 40.701244),1


In [258]:
ssshp['precinct'].unique()

array([  1,   5,   7,   9,   6,  13,  10,  17,  14,  18, 114,  19,  22,
        20,  25,  23,  24,  28,  40,  26,  41,  43,  32,  45,  30,  44,
        42,  33,  48,  49,  52,  46,  34,  47,  50,  60,  61,  63,  62,
        68,  66,  70,  69,  67,  72,  75, 106,  78,  73,  71,  77,  76,
        81,  79,  88,  84, 123, 122, 121, 120, 113, 102,  83, 104, 103,
        90, 107, 112,  94, 108, 110, 109, 115, 100, 101, 105, 111])

In [259]:
for i in ssshp['precinct'].unique():
    
    locals()['precinct_'+str(i)] = ssshp[ssshp['precinct'] == i ]

In [260]:
# determine the threshold of "unsafety"
score = np.array(ssshp['q-score'])
score.sort()
threshold = score[int(len(score) / 4)]
threshold

21.868757000000002

In [268]:
cs.Precinct.unique()

array(['CITYWIDE', 'total CITYWIDE', '001', 'total nan', '005',
       'total 001', '006', 'total 005', '007', 'total 006', '009',
       'total 007', '010', 'total 009', '013', 'total 010', '014',
       'total 013', '017', 'total 014', '018', 'total 017', '019',
       'total 018', '020', 'total 019', '022', 'total 020', '023',
       'total 022', '024', 'total 023', '025', 'total 024', '026',
       'total 025', '028', 'total 026', '030', 'total 028', '032',
       'total 030', '033', 'total 032', '034', 'total 033', '040',
       'total 034', '041', 'total 040', '042', 'total 041', '043',
       'total 042', '044', 'total 043', '045', 'total 044', '046',
       'total 045', '047', 'total 046', '048', 'total 047', '049',
       'total 048', '050', 'total 049', '052', 'total 050', '060',
       'total 052', '061', 'total 060', '062', 'total 061', '063',
       'total 062', '066', 'total 063', '067', 'total 066', '068',
       'total 067', '069', 'total 068', '070', 'total 069', '071'

In [261]:
cs_11['index'] = cs_11.index.astype('int')

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [263]:
cs_11.head()

,Precinct,Violation,CY2011,index,drop
0,CITYWIDE,ACCEPT ON HAIL,4223,0,1
1,CITYWIDE,AFTER HOURS CONSUMPTION,370,1,0
2,CITYWIDE,AGGRAVATED UNLICENSED OPERATOR,549,2,0
3,CITYWIDE,ANGLE PARKING,0,3,0
4,CITYWIDE,BARE HAND CONTACT (FOOD),254,4,0


In [265]:
# mark the total crime statistics, then drop them
def delete_total(x):
    if int(x['index']) % 249 == 0:
        return 1
    else:
        return 0
cs_11['drop'] = cs_11.apply(delete_total, axis=1)   
cs_11 = cs_11[(cs_11['drop'] == 0) & (cs_11['Precinct'] != 'Citywide Total') & (cs_11['Precinct'] != 'CITYWIDE')]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [211]:
ssshp['precinct'].unique()

array([  1,   5,   7,   9,   6,  13,  10,  17,  14,  18, 114,  19,  22,
        20,  25,  23,  24,  28,  40,  26,  41,  43,  32,  45,  30,  44,
        42,  33,  48,  49,  52,  46,  34,  47,  50,  60,  61,  63,  62,
        68,  66,  70,  69,  67,  72,  75, 106,  78,  73,  71,  77,  76,
        81,  79,  88,  84, 123, 122, 121, 120, 113, 102,  83, 104, 103,
        90, 107, 112,  94, 108, 110, 109, 115, 100, 101, 105, 111])

In [266]:
cs_11.head()

,Precinct,Violation,CY2011,index,drop
250,001,ACCEPT ON HAIL,57,250,0
251,001,AFTER HOURS CONSUMPTION,2,251,0
252,001,AGGRAVATED UNLICENSED OPERATOR,0,252,0
253,001,ANGLE PARKING,0,253,0
254,001,BARE HAND CONTACT (FOOD),0,254,0


In [213]:
len(cs_11[cs_11['precinct'] == 1])

0

In [208]:
cs_11['CY2011'].values.shape

(19421,)

In [209]:
19421 / 248

78.31048387096774